In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

In [2]:
data = pd.read_csv('Data/Processed/processed_data.csv')
data.head()

,Unnamed: 0,ticket_id,day_of_week_num,company_size_cat,industry_cat,customer_tier_cat,past_30d_tickets,past_90d_incidents,product_area_cat,customers_affected,error_rate_pct,downtime_min,payment_impact_flag,security_incident_flag,data_loss_flag,priority_cat
0,0,1000000000,3,1,7,1,2,0,3,2,5.451201,6,0,0,0,1
1,1,1000000001,6,1,5,1,2,3,5,24,2.495538,2,0,0,0,1
2,2,1000000002,1,1,4,1,3,1,6,1,3.328402,0,0,0,0,1
3,3,1000000003,3,1,7,2,2,2,5,25,2.931906,16,0,0,0,2
4,4,1000000004,1,1,2,2,5,1,5,28,3.255222,6,0,0,0,1


In [3]:
def percentile_calc(data, column, x):
    return data[column].quantile(x)

In [5]:
thresholds = {
    'customers_p90': data['customers_affected'].quantile(0.90),
    'error_rate_p90': data['error_rate_pct'].quantile(0.90),
    'downtime_p95': data['downtime_min'].quantile(0.95),
    'downtime_p90': data['downtime_min'].quantile(0.90),
    'downtime_p75': data['downtime_min'].quantile(0.75),
}

In [6]:
def sla_tier_assignment(data, thresholds):
    if (
        data['priority_cat'] == 3 and
        (
            data['security_incident_flag'] == 1 or
            data['data_loss_flag'] == 1 or
            data['payment_impact_flag'] == 1
        )
    ):
        return 'SLA-0'

    elif (
        data['customer_tier_cat'] == 3 and
        data['downtime_min'] >= thresholds['downtime_p95']
    ):
        return 'SLA-0'

    elif (
        data['priority_cat'] == 3 or
        data['customers_affected'] >= thresholds['customers_p90'] or
        data['error_rate_pct'] >= thresholds['error_rate_p90'] or
        data['customer_tier_cat'] == 3 or
        data['downtime_min'] >= thresholds['downtime_p90']
    ):
        return 'SLA-1'

    elif (
        data['priority_cat'] == 2 or
        data['downtime_min'] >= thresholds['downtime_p75']
    ):
        return 'SLA-2'

    else:
        return 'SLA-3'


In [7]:
data['SLA-Tier'] = data.apply(
    sla_tier_assignment,
    axis=1, thresholds = thresholds)

In [8]:
def sla_pressure_calc(row):
    if row['SLA-Tier'] == "SLA-0":
        return 1.0
    elif row['SLA-Tier'] == "SLA-1":
        return 1/4
    elif row['SLA-Tier'] == "SLA-2":
        return 1/8
    elif row['SLA-Tier'] == "SLA-3":
        return 1/12
    else:
        return 0

In [11]:
scaler = MinMaxScaler()
data['n_customers_affected'] = scaler.fit_transform(data[['customers_affected']])
data['n_error_rate_pct'] = scaler.fit_transform(data[['error_rate_pct']])
data['recent_ticket_score'] = data['past_30d_tickets'] / max(data['past_30d_tickets'])
data['recent_incident_score'] = data['past_90d_incidents'] / max(data['past_90d_incidents'])

In [12]:
def downtime_severiaty(row):
    if row['downtime_min'] <= thresholds['downtime_p75']:
        return 0.3
    elif row['downtime_min'] <=thresholds['downtime_p90'] and row['downtime_min'] >=thresholds['downtime_p75']:
        return 0.2
    elif row['downtime_min'] >=thresholds['downtime_p90']:
        return 0.3

In [13]:
def incident_flag(row):
    if  row['security_incident_flag'] == 1 or row['data_loss_flag'] == 1 or row['payment_impact_flag'] == 1:
        return 1
    else:
        return 0

In [14]:
def impact_scorer(row):
    downtime_factor = downtime_severiaty(row)
    incident_factor = incident_flag(row)
    customer_factor = row['n_customers_affected']
    error_factor = row['n_error_rate_pct']

    impact_score = (downtime_factor) + (customer_factor * 0.25) + (error_factor * 0.15) + (incident_factor * 0.3)
    return impact_score


In [15]:
def customer_weight(row):
    if row['customer_tier_cat'] == 3:
        return 1.5
    elif row['customer_tier_cat'] == 2:
        return 1.2
    elif row['customer_tier_cat'] == 1:
        return 1

In [16]:
def instability_weight(row):
    instability_factor = (row['recent_ticket_score']*0.6) * (0.4 * row['recent_incident_score'])
    return 1 + (0.5)*instability_factor

In [17]:
def SLA_Risk_Score(row):
    SLA_risk_score = customer_weight(row) * instability_weight(row) * impact_scorer(row) * sla_pressure_calc(row)
    return SLA_risk_score

In [18]:
data['SLA_Risk_Score'] = data.apply(SLA_Risk_Score, axis=1)

In [20]:
data.to_csv("Data/Processed/sla_risk.csv")